# Práctica 1 - Naive Bayes

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García

## Setup

### Importaciones

In [8]:
from Clasificador import Clasificador, ClasificadorNaiveBayes, ClasificadorNaiveBayesScikit
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB

import pandas as pd

### Estrategias de particionado

In [9]:
validacion_cruzada = ValidacionCruzada(numeroParticiones=5)
validacion_simple = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)

### Clasificador generico

In [10]:
clasificador = Clasificador()

### Utilerías

In [11]:
def print_resultados(titulo, resultados):
    print(titulo)
    print("Error promedio: ", resultados[0])
    print("Desviación estándar: ", resultados[1])

def corre_validacion(data, classifier, with_laplace=False):
    validation = clasificador.validacion(
        validacion_cruzada if with_laplace else validacion_simple,
        data,
        ClasificadorNaiveBayes(con_laplace=with_laplace)
    )
    return validation

## Naive Bayes Propio

### Entrenamiento y clasificación para dataset __heart__

In [12]:
datos_heart = Datos("heart.csv")

In [13]:
heart_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", heart_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.1461095272036113
Desviación estándar:  0.05351009204304014


In [14]:
heart_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", heart_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.1461095272036113
Desviación estándar:  0.053510092043040136


In [15]:
heart_validacion_simple = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", heart_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.14036363636363638
Desviación estándar:  0.019048437605394635


In [16]:
heart_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", heart_validacion_simple_laplace)

Validación simple con corrección de Laplace
Error promedio:  0.1410909090909091
Desviación estándar:  0.01970358863148231


### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [17]:
datos_tic_tac_toe = Datos("tic-tac-toe.csv")

In [18]:
tic_tac_toe_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", tic_tac_toe_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.13626679636464206
Desviación estándar:  0.2732406767585417


In [19]:
tic_tac_toe_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", tic_tac_toe_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.13937287214259148
Desviación estándar:  0.27952341003988734


In [20]:
tic_tac_toe_validacion_simple = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", tic_tac_toe_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.2738633723999577
Desviación estándar:  0.035642176772520295


In [40]:
tic_tac_toe_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", tic_tac_toe_validacion_simple_laplace)

Validación simple con corrección de Laplace
Error promedio:  0.2934653151726322
Desviación estándar:  0.03339844252630369


### Analisis de resultados

In [22]:
resultados_naive_bayes_propio = pd.DataFrame(
    [
        (
            "tic-tac-toe",
            tic_tac_toe_validacion_cruzada[0],
            tic_tac_toe_validacion_cruzada[1],
            tic_tac_toe_validacion_cruzada_laplace[0],
            tic_tac_toe_validacion_cruzada_laplace[1],
            tic_tac_toe_validacion_simple[0],
            tic_tac_toe_validacion_simple[1],
            tic_tac_toe_validacion_simple_laplace[0],
            tic_tac_toe_validacion_simple_laplace[1],
        )
    ],
    columns=[
        "Dataset",
        "Promedio error validacion cruzada",
        "Desviación estandar error validación cruzada",
        "Promedio error validacion cruzada con Laplace",
        "Desviacion estandar error validacion cruzada con laplace",
        "Promedio error validacion simple",
        "Desviacion estandar error validacion simple",
        "Promedio error validacion simple con laplace",
        "Desviacion estandar error validacion simple con laplace",
    ],
)

In [23]:
resultados_naive_bayes_propio

,Dataset,Promedio error validacion cruzada,Desviación estandar error validación cruzada,Promedio error validacion cruzada con Laplace,Desviacion estandar error validacion cruzada con laplace,Promedio error validacion simple,Desviacion estandar error validacion simple,Promedio error validacion simple con laplace,Desviacion estandar error validacion simple con laplace
0,tic-tac-toe,0.136267,0.273241,0.139373,0.279523,0.273863,0.035642,0.281704,0.036064


# Naive Bayes con Scikit

### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [39]:
tic_tac_toe_validacion_multinomial = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(MultinomialNB, False)
)

print_resultados("Validación con MultinomialNB sin oneHotEncoder", tic_tac_toe_validacion_multinomial)

AttributeError: 'DataFrame' object has no attribute '_validate_params'